In [1]:
from kim_tools import AFLOW,CRYSTAL_GENOME_INITIAL_STRUCTURES
# 7 : Hexagonal primitive AgO that gives identically oriented (i.e. devoid of the 60* rotation they're supposed to have), should fail
# 1467 : OTi in alpha quartz (sg 152). Unchanged, should pass
from ase.spacegroup import Spacegroup
from ase.io import read
import numpy as np
from scipy.spatial.transform import Rotation 

proto_poscar = 'output/tmp_proto.POSCAR'
spglib_poscar = 'output/tmp_spglib.POSCAR'
aflow = AFLOW(np=19)


In [3]:
import warnings
warnings.filterwarnings("ignore")

structure = CRYSTAL_GENOME_INITIAL_STRUCTURES[3]

print(structure)

parameter_set = structure['parameter_sets'][0]
prototype_label = structure['prototype_label']
parameter_values = parameter_set['parameter_values']


atoms = aflow.build_atoms_from_prototype(structure['species'],prototype_label,parameter_values,proto_file=proto_poscar,verbose=False)
atoms.write(spglib_poscar,format='vasp',sort=True)
space_group = int(prototype_label.split('_')[2])

_,cart_rot,frac_trans_with_ones = aflow.get_basistransformation_rotation_originshift_from_poscars(spglib_poscar,proto_poscar)

print('\nTRANSFORMATION FROM --proto TO SPGLIB:')

print('\nROTATION (CARTESIAN):\n')
print(np.array_str(cart_rot,suppress_small=True))

frac_trans = []
for component in frac_trans_with_ones:
    if np.allclose(component,-1.0) or np.allclose(component,1.0): # sometimes can happen in aflow for some reason
        frac_trans.append(0.)
    else:
        frac_trans.append(component)

frac_trans = np.asarray(frac_trans)

if prototype_label.split('_')[1][0]=='h':
    # we expect to have a 60 degree rotation going from proto to spglib, so undo it
    cart_rot = np.matmul(cart_rot,Rotation.from_rotvec([0,0,-60],True).as_matrix())
    print('\nROTATION ACCOUNTING FOR HEXAGONAL CELL CHANGE (CARTESIAN):\n')
    print(np.array_str(cart_rot,suppress_small=True))
    
# I think using the transformed cell is fine here, since we are assuming the cells are identical
cryst_rot = np.matmul(np.linalg.inv(atoms.cell.T),np.matmul(cart_rot,atoms.cell.T))

print('\nROTATION (CELL)):\n')
print(np.array_str(cryst_rot,suppress_small=True))

print('\nTRANSLATION (CELL):\n')
print(np.array_str(frac_trans,suppress_small=True))

found_match = False
for symop in Spacegroup(space_group).get_symop():
    if np.allclose(cryst_rot,symop[0]) and np.allclose(frac_trans,symop[1]):
        print('\n!!! PASS, OPERATION FOUND IN SPACE GROUP %d !!!\n'%space_group)
        found_match = True
        break
if not found_match:
    print('\n!!! FAIL, OPERATION NOT FOUND IN SPACE GROUP %d !!!\n'%space_group)

{'species': ['Ag', 'O'], 'prototype_label': 'A2B_cP6_224_b_a', 'parameter_names': ['a'], 'modeltype': 'standard', 'version': '002', 'kimnum': '603633842937', 'num_param_sets': '1', 'parameter_sets': [{'library_prototype_label': 'A2B_cP6_224_b_a', 'short_name': 'Cuprite', 'parameter_values': [4.8792], 'auids': ['aflow:0586f177ad1b8870', 'aflow:4d38542810be0c93', 'aflow:808dafa8c6184c07', 'aflow:9017e82281c8a57f', 'aflow:1bfebc242d623030', 'aflow:c4ea5bc1f4d6f01e', 'aflow:cb203d563f80e734', 'aflow:e8eb04c7f40f0f17', 'aflow:eb2fa277fe91d842', 'aflow:f024f2b3bfd420b0', 'aflow:2ac7a746dbb1ec8c', 'aflow:31947a49a0b028ed', 'aflow:3a5d3d4aa9062675'], 'url': 'http://aflowlib.duke.edu/AFLOWDATA/ICSD_WEB/CUB/Ag2O1_ICSD_174090/CONTCAR.relax.vasp'}], 'kim_user_id': '4ad03136-ed7f-4316-b586-1e94ccceb311'}


/home/ilia/.local/lib/python3.10/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['std_lattice']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/ilia/.local/lib/python3.10/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['transformation_matrix']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/ilia/.local/lib/python3.10/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['std_rotation_matrix']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/ilia/.local/lib/python3.10/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['std_positions']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/ilia/.local/lib/python3.10/site-packages/spgli


TRANSFORMATION FROM --proto TO SPGLIB:

ROTATION (CARTESIAN):

[[1 0 0]
 [0 1 0]
 [0 0 1]]

ROTATION (CELL)):

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

TRANSLATION (CELL):

[0 0 0]

!!! PASS, OPERATION FOUND IN SPACE GROUP 224 !!!



In [81]:
atoms=read('output/tmp_proto.POSCAR')

cart_rot=Rotation.from_rotvec([0,0,60],True)


cart_rot=cart_rot.as_matrix()
print(cart_rot)
print(np.array_str(cart_rot,suppress_small=True))

cryst_rot = np.matmul(np.linalg.inv(atoms.cell.T),np.matmul(cart_rot,atoms.cell.T))

print(np.array_str(cart_rot,suppress_small=True))


[[ 0.5       -0.8660254  0.       ]
 [ 0.8660254  0.5        0.       ]
 [ 0.         0.         1.       ]]
[[ 2.82655     2.82655     0.        ]
 [-4.89572821  4.89572821  0.        ]
 [ 0.          0.          6.17289972]]
[[ 1. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]


In [6]:
import warnings
warnings.filterwarnings("ignore")


for i,structure in enumerate(CRYSTAL_GENOME_INITIAL_STRUCTURES):
    print('Processing structure %d\r'%i,end="")
    for parameter_set in structure['parameter_sets']:
        prototype_label = structure['prototype_label']
        parameter_values = parameter_set['parameter_values']
        atoms = aflow.build_atoms_from_prototype(structure['species'],prototype_label,parameter_values,proto_file=proto_poscar,verbose=False)
        proto_dict = aflow.get_prototype(proto_poscar)
        regenerated_prototype_label=proto_dict['aflow_prototype_label']
        if prototype_label == regenerated_prototype_label:
            regenerated_parameter_values = proto_dict['aflow_prototype_params_values']
            if not np.allclose(parameter_values,regenerated_parameter_values):
                print('Parameter values changed for line %d, prototype label %s:\n'%(i+4,prototype_label) + \
                      'Old parameter values: %s\nNew parameter values: %s'%(parameter_values,regenerated_parameter_values))
                break
        else:
            print('Prototype label changed for line %d, prototype label %s:\n'%(i+4,prototype_label) + \
                    'New prototype label: ' + regenerated_prototype_label)
        

Parameter values changed for line 10, prototype label A2B3_oF40_43_b_ab:
Old parameter values: [13.0502, 0.81831696, 0.28929825, 0.16699182, 0.54684454, 0.13337985, 0.99871122, 0.91732275, 0.23370296, 0.38239287]
New parameter values: [10.6792, 1.2220204, 0.35352835, 0.91699182, 0.61662015, 0.79684454, 0.49871122, 0.51629704, 0.16732275, 0.88239287]
Parameter values changed for line 11, prototype label A3B_hP8_162_k_c:
Old parameter values: [5.4927, 0.90287108, 0.69263322, 0.72688523]
New parameter values: [5.4927, 0.90287108, 0.30736678, 0.72688523]
Parameter values changed for line 12, prototype label A3B4_mP14_14_ae_2e:
Old parameter values: [5.9032, 1.5834293, 0.97636875, 142.9688, 0.71032317, 0.64231624, 0.7602652, 0.52075613, 0.54790584, 0.29360513, 0.95922635, 0.81066349, 0.78473]
New parameter values: [3.7073, 2.5213228, 1.5546894, 73.5292, 0.28967683, 0.64231624, 0.95005797, 0.47924387, 0.54790584, 0.227151, 0.04077365, 0.81066349, 0.17449635]
Prototype label changed for line 

KeyboardInterrupt: 